In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir("/content/gdrive/My Drive/BTP-SEM-7/vqa/training")
import sys  
sys.path.append('../') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
from keras.models import Sequential
import pandas as pd
from keras.optimizers import Adam,SGD
from keras.layers import concatenate, Multiply, Lambda, Embedding, Bidirectional, LSTM, GRU,Input, Add,Dense, Conv2D, MaxPooling2D, AveragePooling2D, ZeroPadding2D, Dropout, Flatten, merge, Reshape, Activation
from sklearn.metrics import log_loss
from keras.layers.normalization import BatchNormalization
from keras.models import load_model,Model
from keras.initializers import glorot_uniform
from keras.applications.vgg16 import VGG16
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam, RMSprop
from keras.engine.topology import Layer
from keras import backend as K
from keras import initializers, regularizers, constraints
import cv2
import numpy as np
from keras.callbacks import Callback
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
import glob
from sklearn import preprocessing
from keras.utils import to_categorical,Sequence
from model.model import vqaModelSimple,vqaModelBiLSTM,vqaModelSimpleFeatures,vqaModelBiLSTMFeatures
from keras.callbacks import ModelCheckpoint
import codecs
import h5py
from utils.utils import *
import zipfile

Using TensorFlow backend.


In [0]:
!nvidia-smi

Sat Dec 14 12:48:47 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
df_train = pd.read_csv('../dataset/train_final.csv')
df_val = pd.read_csv("../dataset/val_final.csv")

In [0]:
#Get 60 examples from each class
# df_new = pd.DataFrame(columns=df_train.columns)
# for label in arr_labels:
#   df_temp =  df_train[df_train['multiple_choice_answer']==label]
#   df_temp = df_temp.sample(60,random_state=42,replace=True)
#   df_new = df_new.append(df_temp)
# df_train = df_new

In [0]:
# df_train = df_train.loc[(df_train['multiple_choice_answer']=='yes') | (df_train['multiple_choice_answer']=='no')]
# df_val = df_val[(df_val['multiple_choice_answer']=='yes') | (df_val['multiple_choice_answer']=='no')]

In [0]:
labels = df_train['multiple_choice_answer']
arr_labels = labels.unique()
le = preprocessing.LabelEncoder()
le.fit(arr_labels)
y_train = le.transform(df_train['multiple_choice_answer'])
y_train = to_categorical(y_train)
y_val = le.transform(df_val['multiple_choice_answer'])
y_val = to_categorical(y_val)

In [0]:
#Save label encoder
saveToPickle('../extras/', 'labelEncoder', le)

In [0]:
#Load train and validation features
train_features = readFromPickle('../image_features/', 'vgg16featuresTrain')
val_features = readFromPickle('../image_features/','vgg16featuresVal')

In [0]:
#To caluclate number of words in sentences
# count = df1['questions'].str.split().apply(len).value_counts()
# count 

In [0]:
#Load word embeddings
embedding_index = loadWordEmbeddings('../dataset/glove.840B.300d.txt', embed_size = 300)

In [0]:
image_feature_dims = 4096
time_steps = 20
embed_size = 300
units = 256
num_classes = 1000
# dropout = 0.01
dropout = 0.2

In [8]:
#Creating Tokenizer
df_trainTemp = pd.read_csv('../dataset/train_final.csv')
tokenizer, word_index = createTokenizer(text = df_trainTemp['questions'], max_nb_words = 100000, oov_token = True)

tokenizing input data...
dictionary size:  12487


In [0]:
#Save Tokenizer
saveToPickle('../extras/', 'tokenizer', tokenizer)

In [0]:
# embedding_matrix, words_not_found = createEmbeddingMatrix(word_index, embedding_index,embed_size)
print('preparing embedding matrix...')
words_not_found = []
nb_words = len(word_index)+1#vocabsize
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    
    embedding_vector = embedding_index.get(word)
    if (embedding_vector is not None):
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        print (word)
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

In [0]:
#Save embedding matrix
saveToH5Py(embedding_matrix, '../dataset/' , 'embedding_matrix_glove')

In [0]:
#Read embedding matrix
embedding_matrix = readFromH5Py('../dataset/', 'embedding_matrix_glove')

In [0]:
#Create encoded and padded training data
X_train_sequences, X_train_questions = createEncodedPaddedText(tokenizer, df_train['questions'],time_steps=time_steps)

#Create encoded and padded validation data
X_val_sequences, X_val_questions = createEncodedPaddedText(tokenizer, df_val['questions'],time_steps=time_steps)

In [0]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Supports Masking.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.supports_masking = True
        #self.init = initializations.get('glorot_uniform')
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        # eij = K.dot(x, self.W) TF backend doesn't support it

        # features_dim = self.W.shape[0]
        # step_dim = x._keras_shape[1]

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
    #print weigthted_input.shape
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        #return input_shape[0], input_shape[-1]
        return input_shape[0],  self.features_dim

In [0]:
def vqaModelBiLSTMAttFeatures(embedding_matrix, image_feature_dims=4096, trainable=False, num_classes=1000,embed_size=300, vocab_size=10000, time_steps=20, unit_length=512, dropout=0.5):
    #LSTM
    inputsLSTM = Input(shape=(time_steps,))
    x = Embedding(output_dim=embed_size, input_dim=vocab_size, weights=[embedding_matrix],input_length=time_steps, trainable=trainable)(inputsLSTM)
    # default merge_mode is concatenate others are mul,ave,sum
    y= Bidirectional(LSTM(unit_length, return_sequences=True), input_shape=(time_steps, 1), merge_mode='mul')(x)
    a= Bidirectional(LSTM(unit_length, return_sequences=True), input_shape=(time_steps, 1), merge_mode='sum')(y)
    att = Attention(time_steps)(a)
    modelLSTM = Model(inputs=inputsLSTM, outputs=att)

    #Image MODEL
    inputsImage = Input(shape=(image_feature_dims,))
    outputsImage = Lambda(lambda  x: K.l2_normalize(x,axis=1))(inputsImage)
    modelVGG16New = Model(inputsImage, outputsImage)

    merged = concatenate([modelLSTM.output, modelVGG16New.output])
    dense1 = Dense(1000, activation='tanh')(merged)
    dense1 = Dropout(dropout)(dense1)
    output =  Dense(num_classes, activation='softmax')(dense1)
    model = Model(inputs=[modelLSTM.input,modelVGG16New.input], outputs=output)
    return model


In [0]:
#MODEL 2
def vqaModelAttention(embedding_matrix, vdim = 2048, qdim=512, num_hid = 1024, image_feature_dims=2048, trainable=False, num_classes=1000,embed_size=300, vocab_size=10000, time_steps=14, unit_length=512, dropout=0.5, k=36):
    #FC model
    #inputFC = Input(shape=(vdim+qdim,))
    # denseFC = Dense(num_hid, activation='relu')(inputFC)
    # denseFC = Dense(1, activation='relu')(denseFC)
    #question
    inputsGRU = Input(shape=(time_steps,))
    x = Embedding(output_dim=embed_size, input_dim=vocab_size, weights=[embedding_matrix],input_length=time_steps, trainable=trainable)(inputsGRU)
    y= GRU(unit_length, return_sequences=False)(x)
    dense=Dense(512,activation='tanh')(y)
    y_new = Lambda(lambda x: K.expand_dims(x,axis=1))(dense)

    # y_new = K.expand_dims(y,axis=1)
    expanded_qdim = Lambda(lambda x: K.repeat_elements(x,k,axis=1))(y_new)
    # expanded_qdim= K.repeat_elements(y_new,k,axis=1)
    #Image MODEL (assuming y1)
    
    y1 = Input(shape=(k,2048,))

    concat = concatenate([expanded_qdim, y1],axis=2)
    denseFC = Dense(num_hid, activation='relu')(concat)
    denseFC = Dense(1, activation='softmax')(denseFC)
    #FCModel = Model(inputs=concat, outputs=denseFC)

    afunc =  Dense(1, activation='softmax')(denseFC)
    afunc1 = Lambda(lambda x: K.repeat_elements(x,2048,axis=2))(afunc)
    # afunc1= K.repeat_elements(afunc,2048,axis=1)
    
    res= Multiply()([afunc1,y1])
    res = Lambda(lambda x: K.sum(res,axis=1))(res)
    # res= K.sum(res,axis=0)
    dense3= Dense(512, activation='tanh')(res)
    combined= Multiply()([dense, dense3])
    dense4= Dense(512, activation='tanh')(combined)
    dense5= Dense(512, activation='tanh')(dense4)
    dense6 = Dropout(dropout)(dense5)
    output =  Dense(num_classes, activation='softmax')(dense6)


    model = Model(inputs=[inputsGRU, y1], outputs=output)
    return model

In [0]:
#TESTING ATTENTION MODEL
vqa_model = vqaModelAttention(embedding_matrix=embedding_matrix, image_feature_dims =2048, trainable=False, num_classes=num_classes,embed_size=embed_size, vocab_size=len(word_index)+1, time_steps=time_steps, unit_length=512, dropout=dropout, k=36)

In [0]:
vqa_model = vqaModelBiLSTMAttFeatures(embedding_matrix=embedding_matrix, image_feature_dims = image_feature_dims, trainable=False, num_classes=num_classes,embed_size=embed_size, vocab_size=len(word_index)+1, time_steps=time_steps, unit_length=units, dropout=dropout)
# vqa_model = vqaModelBiLSTMFeatures(embedding_matrix=embedding_matrix, image_feature_dims = image_feature_dims, trainable=False, num_classes=num_classes,embed_size=embed_size, vocab_size=len(word_index)+1, time_steps=time_steps, unit_length=units, dropout=dropout)
# vqa_model = vqaModelSimpleFeatures(embedding_matrix=embedding_matrix, image_feature_dims = image_feature_dims, trainable=False, num_classes=num_classes,embed_size=embed_size, vocab_size=len(word_index)+1, time_steps=time_steps, unit_length=units, dropout=dropout)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [112]:
vqa_model.summary()

Model: "model_27"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_91 (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_42 (Embedding)        (None, 20, 300)      3746400     input_91[0][0]                   
__________________________________________________________________________________________________
gru_42 (GRU)                    (None, 512)          1248768     embedding_42[0][0]               
__________________________________________________________________________________________________
dense_123 (Dense)               (None, 512)          262656      gru_42[0][0]                     
___________________________________________________________________________________________

In [0]:
#Save model
# serialize model to JSON
model_json = vqa_model.to_json()
with open('../extras/vqaModelBiLSTMFeatures.json', "w") as json_file:
    json_file.write(model_json)

In [0]:
#Skip files not present
class My_Custom_Generator(Sequence) :
  
  def __init__(self,questions, image_filenames, labels, batch_size,type_of_data, archive) :
    self.questions = questions
    self.image_filenames = image_filenames
    self.labels = labels
    self.batch_size = batch_size
    self.type_of_data = type_of_data
    self.archive = archive
    
    
  def __len__(self) :
    return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
    questions_x = self.questions[idx * self.batch_size : (idx+1) * self.batch_size]
    questions = []
    images_x = []
    labels_y = []
    for counter, img in enumerate(batch_x):
      try:
        zeros = 12 - len(str(img))
        if(self.type_of_data=='val'):
          file_name = 'COCO_val2014_' + '0'*zeros + str(img) + '.jpg'
          imgData = self.archive.read('val2014/' + file_name)
          im = cv2.imdecode(np.frombuffer(imgData, np.uint8), 1)
  #         im = cv2.imread('../dataset/images/validation/val2014/' + file_name)
        if(self.type_of_data=='train'):
          file_name = 'COCO_train2014_' + '0'*zeros + str(img) + '.jpg'
          imgData = self.archive.read('train2014/' + file_name)
          im = cv2.imdecode(np.frombuffer(imgData, np.uint8), 1)
  #         im = cv2.imread('../dataset/images/train/train2014/' + file_name)
        im = cv2.resize(im, (224,224))
        im = im/255.0
        images_x.append(im)
        labels_y.append(batch_y[counter])
        questions.append(questions_x[counter])
#         print(counter)
      except:
        print(counter)
        continue
    
    return [np.asarray(questions), np.asarray(images_x)], np.asarray(labels_y)
#     return np.array([
#             resize(imread('/content/all_images/' + str(file_name)), (80, 80, 3))
#                for file_name in batch_x])/255.0, np.array(batch_y)

In [0]:
#For Image preloaded image features
class My_Custom_Generator_Features(Sequence) :
  
  def __init__(self,questions, image_filenames, labels, batch_size,dictionary) :
    self.questions = questions
    self.image_filenames = image_filenames
    self.labels = labels
    self.batch_size = batch_size
    self.dictionary = dictionary
    
    
  def __len__(self) :
    return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
  
  
  def __getitem__(self, idx) :
    batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
    questions_x = self.questions[idx * self.batch_size : (idx+1) * self.batch_size]
    images_x = []
    for counter, img in enumerate(batch_x):
      images_x.append(self.dictionary[img])
    
    return [np.asarray(questions_x), np.asarray(images_x)], np.asarray(batch_y)
#     return np.array([
#             resize(imread('/content/all_images/' + str(file_name)), (80, 80, 3))
#                for file_name in batch_x])/255.0, np.array(batch_y)

In [0]:
#PreLoaded Image Features
batch_size = 512
my_training_batch_generator = My_Custom_Generator_Features(X_train_questions, df_train['image_id'], y_train, batch_size, train_features)
my_validation_batch_generator = My_Custom_Generator_Features(X_val_questions, df_val['image_id'], y_val, batch_size, val_features)

In [0]:
batch_size = 128
train_archive = zipfile.ZipFile('../../Dataset-Coco-V2/vqa_v2_images.zip','r')
val_archive = zipfile.ZipFile('../../Dataset-Coco-V2/vqa_v2_validation_images.zip','r')
my_training_batch_generator = My_Custom_Generator(X_train_questions, df_train['image_id'], y_train, batch_size, 'train',train_archive)
my_validation_batch_generator = My_Custom_Generator(X_val_questions, df_val['image_id'][:5120], y_val[:5120], batch_size, 'val',val_archive)

In [0]:
optimizer = Adam(lr=0.001)
# optimizer = RMSprop()
vqa_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
vqa_model.load_weights('../weights/best_weights.h5')

In [0]:
checkpoint = ModelCheckpoint('../weights/best_weights.h5', monitor='val_acc', verbose=1, save_best_only=True,save_weights_only=True, mode='max')
callbacks_list = [checkpoint]
vqa_model.fit_generator(generator=my_training_batch_generator,
                   epochs = 55,
                   verbose = 1,
                   validation_data = my_validation_batch_generator,
                   callbacks = callbacks_list)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/55


In [0]:

vqa_model.save_weights('../weights/vqaModelBiLSTM_5epochs.h5')

In [0]:
cntr=0
lst=val_archive.namelist()
for img in df_val['image_id'][:5000]:
  zeros = 12 - len(str(img))
  file_name = 'val2014/' +'COCO_val2014_' + '0'*zeros + str(img) + '.jpg'
  if(file_name not in lst):
    cntr=cntr+1
    
print(cntr)

0
